## **Finetuning** `facebook/bart-large` **on** `lmsys/chatbot_arena_conversations` **dataset**

---

In [1]:
# !pip install datasets transformers torch

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name= "facebook/bart-large"

model= AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer= AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces = True)

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [22]:
# Ensure model is on CUDA (GPU)
device = model.device
texts = ["This is an example sentence.", "Another sentence to check dimensions."]
# Tokenize and move inputs to the same device as the model
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# Move input tensors to the correct device (same as model)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Forward pass with output_hidden_states=True to include decoder hidden states
outputs = model(**inputs, output_hidden_states=True)

# Check output shapes
print(f"Logits shape: {outputs.logits.shape}")
print(f"Encoder hidden states shape: {outputs.encoder_last_hidden_state.shape}")
print(f"Decoder hidden states shape: {outputs.decoder_hidden_states[0].shape}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


So, before I access the dataset, the dataset is a gated dataset. i.e., it doesnot allow to access until and unless users have authorized and agreed to terms and conditions. Hence Huggingface login is required.

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Hugging_face")
secret_value_1 = user_secrets.get_secret("wandb_api_key")

In [5]:
from huggingface_hub import login

login(secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
from datasets import load_dataset

ds = load_dataset("lmsys/chatbot_arena_conversations")

README.md:   0%|          | 0.00/7.00k [00:00<?, ?B/s]

(…)-00000-of-00001-cced8514c7ed782a.parquet:   0%|          | 0.00/41.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/33000 [00:00<?, ? examples/s]

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['question_id', 'model_a', 'model_b', 'winner', 'judge', 'conversation_a', 'conversation_b', 'turn', 'anony', 'language', 'tstamp', 'openai_moderation', 'toxic_chat_tag'],
        num_rows: 33000
    })
})

In [8]:
import pandas as pd

df = pd.DataFrame(ds['train'])

In [9]:
df['language'].unique()

array(['English', 'unknown', 'French', 'Latin', 'Arabic', 'Italian',
       'Latvian', 'Portuguese', 'Japanese', 'German', 'Spanish',
       'Chinese', 'Russian', 'Slovak', 'Turkish', 'Akan', 'Danish',
       'Uzbek', 'Esperanto', 'Scots', 'Indonesian', 'Hebrew', 'Dutch',
       'Korean', 'Corsican', 'Wolof', 'Waray', 'Luxembourgish',
       'Bulgarian', 'Serbian', 'Czech', 'Catalan', 'Manx', 'Swedish',
       'Malagasy', 'Polish', 'Norwegian', 'Interlingua', 'Oromo',
       'Tswana', 'Finnish', 'Maori', 'Tsonga', 'Romanian', 'Bislama',
       'Welsh', 'Xhosa', 'Galician', 'Malay', 'Persian', 'Vietnamese',
       'zzp', 'Hawaiian', 'Aymara', 'Norwegian Nynorsk', 'Afrikaans',
       'Icelandic', 'Ukrainian', 'Occitan', 'Hungarian', 'Thai',
       'Lithuanian', 'Croatian', 'Quechua', 'Haitian Creole',
       'Western Frisian', 'Interlingue', 'Somali', 'Slovenian', 'Afar',
       'Irish', 'Kalaallisut', 'Volapük', 'Fijian', 'Sanskrit',
       'Estonian', 'Basque', 'Hmong', 'Macedonian', '

In [10]:
##@ First lets filter out the 'English' ones only:

df_eng = df[df['language'] == 'English']

df_eng['language'].unique()

array(['English'], dtype=object)

In [11]:
df_eng['conversation_a'][2]

[{'content': 'Fuji vs. Nikon, which is better?', 'role': 'user'},
 {'content': 'Both Fuji and Nikon are popular camera brands that offer a wide range of cameras for different types of photography. It really depends on your specific needs and preferences as to which brand is "better" for you.\n\nFujifilm is known for its high image quality, especially in the areas of color accuracy and dynamic range. Many photographers also appreciate the company\'s focus on ergonomics and the overall user experience of its cameras. Additionally, Fujifilm has a loyal following among enthusiasts and professional photographers, thanks to its commitment to film photography and its high-end mirrorless cameras.\n\nNikon, on the other hand, is known for its advanced autofocus system and image stabilization technology, as well as its high-end cameras and lenses. Nikon also has a wide selection of cameras and lenses to choose from, including both DSLR and mirrorless options.\n\nUltimately, it\'s difficult to sa

In [12]:
df_eng['conversation_b'][2]

[{'content': 'Fuji vs. Nikon, which is better?', 'role': 'user'},
 {'content': "This is a subjective question and the answer depends on the individual preferences and needs. Both Fujifilm and Nikon are well-known for their high-quality cameras, and the choice between the two ultimately depends on the user's personal preferences and needs.\n\nSome factors to consider when deciding between a Fuji and a Nikon camera include:\n\n1. Image quality: Fuji cameras are known for their high-quality images, with a distinctive film-like quality. Nikon cameras are also notable for their image quality, but they may not offer the same level of sharpness and color accuracy as Fuji cameras.\n\n2. Focusing: Fuji cameras are known for their fast and precise autofocus, while Nikon cameras tend to have a slower and more manual focus system.\n\n3. Image size: Fuji cameras are known for their wide range of photo sizes, from small compacts to large-format cameras. Nikon cameras are also capable of producing hi

Looks like this dataset has two conversations, "a" and "b".

So, during preprocessing, we could just use "conversation_b"'s user role content ie the questions and pass it to the inputs while targets would be the content from assistant role...

In [13]:
##@ Preprocessing_function 

def preprocess_function(data):

    inputs= [
        " ".join([entry['content'] for entry in conv if entry['role'] == 'user'])
        for conv in data['conversation_b']
    ]

    targets= [
        " ".join([entry['content'] for entry in conv if entry['role'] == 'assistant'])
        for conv in data['conversation_b']
    ]
    
    ## Tokenizing
    model_inputs = tokenizer(inputs, max_length= 128, padding= "max_length", truncation= True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=1028, padding='max_length', truncation=True)

    ## adding labels to inputs 
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

the `df_eng` was in pandas. Now for mapping we need to convert this back to hugging face dataset 

In [14]:
from datasets import Dataset, DatasetDict

hf_dataset = Dataset.from_pandas(df_eng, preserve_index= False)

hf_dataset= hf_dataset.train_test_split(test_size=0.2)

dataset_dict = DatasetDict({
    'train': hf_dataset['train'], 
    'validation': hf_dataset['test']
})

In [15]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['question_id', 'model_a', 'model_b', 'winner', 'judge', 'conversation_a', 'conversation_b', 'turn', 'anony', 'language', 'tstamp', 'openai_moderation', 'toxic_chat_tag'],
        num_rows: 23364
    })
    validation: Dataset({
        features: ['question_id', 'model_a', 'model_b', 'winner', 'judge', 'conversation_a', 'conversation_b', 'turn', 'anony', 'language', 'tstamp', 'openai_moderation', 'toxic_chat_tag'],
        num_rows: 5842
    })
})

In [16]:
tokenized_datasets = dataset_dict.map(preprocess_function, batched= True)

Map:   0%|          | 0/23364 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5842 [00:00<?, ? examples/s]

In [17]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = Seq2SeqTrainer(
    model= model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator= data_collator,
    tokenizer=tokenizer,
)

In [19]:
import wandb
wandb.login(key= secret_value_1)
wandb.init(project="Chatbot_", name="version1")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: firojpaudel (firojpaudel-madan-bhandari-memorial-college). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/parallel_apply.py", line 84, in _worker
    output = module(*input, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/bart/modeling_bart.py", line 1641, in forward
    outputs = self.model(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local